In [6]:
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam

from dataset import MyDataset
from model import Generator, Discriminator, weights_init
from train import train
# from utils import print_image, make_video, get_grid_image

import os
import numpy as np
import matplotlib.pyplot as plt

In [5]:
batch_size = 128
z_size = 100
out_chnl = 3
in_chnl = 3
d_chnl = 32
g_chnl = 32
lr = 0.0005

# dataloader = DataLoader(MyDataset(), batch_size=batch_size, shuffle=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

G = Generator(z_size, g_chnl, out_chnl).to(device)
D = Discriminator(d_chnl, in_chnl).to(device)

G.apply(weights_init)
D.apply(weights_init)

opt_G = Adam(G.parameters(), lr=lr, betas=(0.5, 0.999))
opt_D = Adam(D.parameters(), lr=lr, betas=(0.5, 0.999))

fixed = torch.normal(0,1,size=(100, z_size, 1, 1)).to(device)

In [10]:
checkpoints =  [file for file in os.listdir("./") if file.endswith(".pt")]
if (checkpoints):
    state = torch.load(f"./{checkpoints[0]}")
    G.load_state_dict(state["G"])
    D.load_state_dict(state["D"])
    fixed = state["fixed"]

# train(G, D, opt_G, opt_D, z_size, fixed, dataloader, device, 50)

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
import torch
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

# 이미지 생성 (G는 사전 학습된 Generator라고 가정)
z_size = 100  # latent vector 크기
G.eval()  # eval 모드로 전환

with torch.no_grad():
    z = torch.randn((100, z_size, 1, 1))
    imgs = G(z).cpu()  # GPU 사용 시 .cpu() 붙이기

# 이미지 출력 함수
def show_image(index):
    img = imgs[index]
    img = (img + 1) / 2  # [-1, 1] → [0, 1] 정규화
    np_img = img.permute(1, 2, 0).numpy()  # (C, H, W) → (H, W, C)

    plt.figure(figsize=(3, 3))
    plt.imshow(np_img)
    plt.axis('off')
    plt.title(f"Generated Image #{index}")
    plt.show()

# 슬라이더 위젯 만들기
slider = widgets.IntSlider(value=0, min=0, max=99, step=1, description='index:')
widgets.interact(show_image, index=slider)


interactive(children=(IntSlider(value=0, description='Image:', max=99), Output()), _dom_classes=('widget-inter…

<function __main__.show_image(index)>